<a href="https://colab.research.google.com/github/LoganK64/CSCE_489/blob/main/HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# HW 4

<h3> Task 1: Create a TfidfVectorizer </h3>

In [ ]:
#!pip3 install scikit-learn
#!pip3 install pandas
from __future__ import print_function, division
from datetime import datetime 
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, datasets
from torch.utils.data import DataLoader
import gzip
import tarfile
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from google.colab import files
uploaded = files.upload()
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

Import the dataset from the files given

In [ ]:
dataset = []
y = []
kitchen = []
ham1 = tarfile.open("kitchen-l.tar.gz", "r:gz")
for member in ham1.getmembers():
  f = ham1.extractfile(member)
  if f:
    content = f.read()
    kitchen.append(content)
    #dataset.append(content)

lokay = []
ham2 = tarfile.open("lokay-m.tar.gz", "r:gz")
for member in ham2.getmembers():
  f = ham2.extractfile(member)
  if f:
    content = f.read()
    lokay.append(content)
    #dataset.append(content)


gp = []
gp19 = []
gp1012 = []
spam = tarfile.open("GP.tar.gz", "r:gz")
for member in spam.getmembers():
  f = spam.extractfile(member)
  if f:
    content = f.read()
    gp.append(content)
    #dataset.append(content)
count = 0
divider = len(gp)/12
for item in gp:
  if count < 10 * divider:
    gp19.append(item)
  else:
    gp1012.append(item)
  count+=1

print(len(gp))
print(len(gp19))
print(len(gp1012))
dataset = gp19+kitchen+gp1012+lokay
print(len(dataset))
for i in range(len(dataset)):
  if(i < len(gp19)):
    y.append(1)
  elif(i<len(gp19)+len(kitchen)):
    y.append(0)
  elif(i < len(gp19)+len(kitchen)+len(gp1012)):
    y.append(1)
  else:
    y.append(0)

print(len(y))

13719
11433
2286
20095
20095


Actually declare the TFIDF

In [ ]:
tfIdfVectorizer=TfidfVectorizer(use_idf=True, encoding='utf-8', decode_error='ignore')
tfIdf = tfIdfVectorizer.fit_transform(dataset)
df = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)

In [ ]:
print(tfIdf.shape)

(20095, 413455)


<h3> Task 2: Display Frame Data of Spam and Ham </h3>

Spam TFIDF

In [ ]:
print(df.head(25))

                 TF-IDF
3d             0.482524
hoodia         0.403120
td             0.239992
hoodiashop     0.236128
1039           0.195649
font           0.167399
strong         0.164627
webstat        0.157419
tr             0.152243
serif          0.134902
cactus         0.133807
helvetica      0.131913
map            0.127057
sans           0.121121
width          0.116883
arial          0.106143
face           0.103886
tablets        0.101161
diet           0.098929
20             0.089512
tbody          0.089323
tabs           0.080514
swisspresence  0.078709
table          0.078626
pict           0.075585


Ham TFIDF

In [ ]:
df = pd.DataFrame(tfIdf[len(gp19)+1].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
print(df.head(25))

              TF-IDF
plants      0.270877
cn          0.243667
enron       0.232541
kw          0.226732
wisvest     0.186673
the         0.175430
coal        0.173402
nepool      0.164177
in          0.132049
recipients  0.121573
broderick   0.121515
ou          0.119002
na          0.117725
ct          0.115601
john        0.115035
of          0.113067
llodra      0.112488
duran       0.105440
nrg         0.103180
mitch       0.098417
asset       0.097728
socialized  0.095203
approx      0.093249
robinson    0.093007
to          0.092751


<h3> Task 3: Create a RandomForestClassifier </h3>

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=50)
model.fit(tfIdf[:(len(gp19)+len(kitchen))],y[:(len(gp19)+len(kitchen))])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

<h3> Task 4: Predict the emails in the Test Portion of the data using the RFC </h3>

In [ ]:
from sklearn.metrics import confusion_matrix
ypred = model.predict(tfIdf[(len(gp19)+len(kitchen)):])
confused=confusion_matrix(y[(len(gp19)+len(kitchen)):],ypred)
tn, fp, fn, tp = confused.ravel()
print("TN: ", tn,"FP: ", fp,"FN: ", fn,"TP: ", tp)
print(confused)

TN:  2361 FP:  3 FN:  0 TP:  2286
[[2361    3]
 [   0 2286]]


<h3> Task 5: Make a new RFC using StopWords </h3>

Read in Stopwords List

In [ ]:
file = open("stopwords-en.txt")
stopwords = file.read().split('\n')

["'ll", "'tis", "'twas", "'ve", '10', '39', 'a', "a's", 'able', 'ableabout', 'about', 'above', 'abroad', 'abst', 'accordance', 'according', 'accordingly', 'across', 'act', 'actually', 'ad', 'added', 'adj', 'adopted', 'ae', 'af', 'affected', 'affecting', 'affects', 'after', 'afterwards', 'ag', 'again', 'against', 'ago', 'ah', 'ahead', 'ai', "ain't", 'aint', 'al', 'all', 'allow', 'allows', 'almost', 'alone', 'along', 'alongside', 'already', 'also', 'although', 'always', 'am', 'amid', 'amidst', 'among', 'amongst', 'amoungst', 'amount', 'an', 'and', 'announce', 'another', 'any', 'anybody', 'anyhow', 'anymore', 'anyone', 'anything', 'anyway', 'anyways', 'anywhere', 'ao', 'apart', 'apparently', 'appear', 'appreciate', 'appropriate', 'approximately', 'aq', 'ar', 'are', 'area', 'areas', 'aren', "aren't", 'arent', 'arise', 'around', 'arpa', 'as', 'aside', 'ask', 'asked', 'asking', 'asks', 'associated', 'at', 'au', 'auth', 'available', 'aw', 'away', 'awfully', 'az', 'b', 'ba', 'back', 'backed', 

Make new TFIDF

In [ ]:
tfIdfVectorizer=TfidfVectorizer(use_idf=True, encoding='utf-8', decode_error='ignore', stop_words=stopwords)
tfIdf = tfIdfVectorizer.fit_transform(dataset)
df = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abbr', 'acronym', 'address', 'ain', 'applet', 'article', 'audio', 'base', 'basefont', 'bdo', 'blockquote', 'body', 'button', 'canvas', 'center', 'cite', 'code', 'col', 'colgroup', 'daren', 'datalist', 'dd', 'del', 'dfn', 'div', 'dl', 'doctype', 'dt', 'em', 'embed', 'fieldset', 'figcaption', 'figure', 'font', 'footer', 'form', 'frame', 'frameset', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'hadn', 'head', 'header', 'herse', 'himse', 'iframe', 'img', 'input', 'ins', 'itse', 'kbd', 'label', 'legend', 'link', 'main', 'map', 'mark', 'mayn', 'meta', 'meter', 'mightn', 'mon', 'mustn', 'myse', 'nav', 'needn', 'noscript', 'object', 'ol', 'optgroup', 'option', 'oughtn', 'param', 'pre', 'progress', 'samp', 'script', 'select', 'shan', 'source', 'span', 'strike', 'strong', 'style', 'table', 'tbody', 'textarea',

In [ ]:
print(df.head(25))

                 TF-IDF
3d             0.520018
hoodia         0.434445
hoodiashop     0.254476
1039           0.210852
font           0.180407
strong         0.177420
webstat        0.169651
serif          0.145384
cactus         0.144205
helvetica      0.142163
map            0.136930
sans           0.130533
arial          0.114391
tablets        0.109021
diet           0.106616
20             0.096468
tbody          0.096264
tabs           0.086771
swisspresence  0.084825
table          0.084736
pict           0.081458
coords         0.078147
rect           0.074903
size           0.066027
weight         0.065456


In [ ]:
df = pd.DataFrame(tfIdf[len(gp19)+1].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
print(df.head(25))

              TF-IDF
plants      0.343717
wisvest     0.236870
coal        0.220030
nepool      0.208326
recipients  0.154265
broderick   0.154191
ou          0.151002
ct          0.146687
john        0.145968
llodra      0.142737
duran       0.133794
nrg         0.130925
mitch       0.124881
asset       0.124007
socialized  0.120804
approx      0.118324
robinson    0.118018
conneticut  0.116433
locational  0.110561
congested   0.107428
louise      0.101964
kitchen     0.101450
lavorato    0.100471
dana        0.096059
proposed    0.095716


Make New RFC

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=50)
model.fit(tfIdf[:(len(gp19)+len(kitchen))],y[:(len(gp19)+len(kitchen))])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

Test New RFC

In [ ]:
from sklearn.metrics import confusion_matrix
ypred = model.predict(tfIdf[(len(gp19)+len(kitchen)):])
confused=confusion_matrix(y[(len(gp19)+len(kitchen)):],ypred)
tn, fp, fn, tp = confused.ravel()
print("TN: ", tn,"FP: ", fp,"FN: ", fn,"TP: ", tp)
print(confused)

TN:  2360 FP:  4 FN:  0 TP:  2286
[[2360    4]
 [   0 2286]]
